In [ ]:
#!pip3 install seaborn pandas matplotlib numpy    

every successive data of each device id were in an interval of approx 60 sec

In [ ]:
import csv 
import pandas as pd 
from datetime import date , time 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from datetime import datetime
import warnings
from traceback import print_exc
import datetime
file_names = ["m1.csv" , "m2.csv" , "m3.csv" , "m4.csv"]
save_path = "combined_mahanagar_data.csv" 
rcp = "rush_clasified.csv" #rush_classified_path
dpd = "data_per_day.csv" #data_per_day
save_path_2 = "data_tracking.csv"
columns = ["distance" , "totalDistance" , "deviceId","fixTime","latitude","longitude","speed"]
total_bus , total_dates = [],[]
peak_morning = [8,11]
peak_evening = [16,19]
dates_given = ['2020-02-23', '2019-08-22', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-06', '2020-01-27', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20']
dict_dates = {
        key:0 for key in dates_given 
        
    }
dict_dates_2 = {
    key:[0,0,0] for key in dates_given
}
buses = [130, 2, 131, 132, 133, 134, 135, 136, 200, 137, 73, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 166, 167, 169]
total_buses = {}
total_dates = []
folder_img = "distance_vs_time"
os.makedirs(folder_img, exist_ok=True)
total_entries=[0,0,0]
discarded_entries = [0,0,0]
def make_int(x):
    x=str(x)
    
    y=list(x)
    
    if "-" in y:
        y = x 
        year = int("".join(x[0:4]))
        month = int("".join(x[5:7]))
        day = int("".join(x[8:10]))
        return year , month , day
    else:
        year = int("".join(x[0:2]))
        month = int("".join(x[3:5]))
        day = int("".join(x[6:8]))
        return year , month , day

In [ ]:


    
total_lines = 0
total_discarded = []
def classify_hr(hour):
    """Classifies a time (in absolute seconds) into morning, evening, or free period."""
    if peak_morning[0] <= hour <= peak_morning[1]:
        return 0
    if peak_evening[0] <= hour <= peak_evening[1]:
        return 2
    return 1
def try_float_convert(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        
        return value
with open(save_path ,"w",newline="") as file:
    writer_file = csv.DictWriter(file , fieldnames=columns)
    writer_file.writeheader()
    for name in file_names:
        with open(name ,"r") as csv_file: 
            reader = csv.DictReader(csv_file)
            
            
            
            for line in reader:
                lines_to_write = [
                    try_float_convert(line[cols])  
                    for cols in columns 
                    #if line[cols] not in columns and line["alarm"]!="powerCut"
                    if line[cols] not in columns
                    ]
                try:
                    dict_temp = {
                    columns[0]:lines_to_write[0],
                    columns[1]:lines_to_write[1],
                    columns[2]:int(lines_to_write[2]),
                    columns[3]:lines_to_write[3],
                    columns[4]:lines_to_write[4],
                    columns[5]:lines_to_write[5],
                    columns[6]:lines_to_write[6],
                }
                    writer_file.writerow(dict_temp)
                    dict_temp={}
                    total_lines+=1 
                except:
                    pass 
    file.close()
print(f" a total of {total_lines} were created in {save_path}")
                

In [ ]:


def date_parser(dates):
    listed = []
    i=1
    print(dates[:10])
    for date in dates:
        
        i+=1        
        stinged = list(date)
        
        year = int(float("".join(stinged[0:4])))
        month = int(float("".join(stinged[5:7])))
        day=int(float("".join(stinged[8:10])))
        time1 = time(int(float("".join(stinged[11:13]))),int(float("".join(stinged[14:16]))) , int(float("".join(stinged[17:19]))))
        
        d1=datetime.date(year,month , day)
        
        listed.append(str(datetime.datetime.combine( d1, time1))) 
    return listed
df = pd.read_csv(save_path,header=0)

df["fixTime"] = date_parser(df["fixTime"])
df.to_csv(save_path , index = False)
print("done")

In [ ]:

            
with open(save_path , "r") as file:
    
    lines = csv.DictReader(file)
    temp_recorder = [[0,0,0] for _ in range(0 , 41)]

    

    for line in lines:
        #print(line)
        device_id = int(line["deviceId"])
        indexed = buses.index(device_id)
        
        time1 = line["fixTime"]
        time1 = time1.split() 
        yr , mon ,day = make_int(time1[0]) 
        hr,min,sec =  make_int(time1[1]) 
        # if time1[0] not in dates_given:
        #     dates_given.append(time1[0])
        
        
        
        
        
        #morning_peak 0 , free_hour 1 , evening_peak 2
        hour_time = 0 if peak_morning[0]<=hr<=peak_morning[1] else (1 if peak_morning[1]<hr<peak_evening[0] else (2 if peak_evening[0]<=hr<=peak_evening[1] else 1))
        total_entries[hour_time]+=1 
        temp_recorder[indexed][hour_time]+=1
        dict_dates[time1[0]]+=1
        some_data = dict_dates_2[time1[0]]
        some_data[hour_time]+=1 
        dict_dates_2[time1[0]] = some_data
    file.close()
with open(rcp , "w") as rcp_writer:
    writer = csv.DictWriter(rcp_writer , fieldnames=["device_id" , "morning_peak(7-10)" , "free_hour" , "evening_peak(4-7)"])
    writer.writeheader()
    for i in range(0,41):
        writer.writerow({
            "device_id":buses[i] , 
            "morning_peak(7-10)":temp_recorder[i][0],
            "free_hour":temp_recorder[i][1],
            "evening_peak(4-7)":temp_recorder[i][2]
            })      
    rcp_writer.close()
with open(dpd , "w") as dpd_writer:
    writer = csv.DictWriter(dpd_writer , fieldnames = ["date" ,"data_count" , "morning_rush" , "free" , "evening_rush"])
    writer.writeheader()
    for key,value in dict_dates.items():
        extra_val = dict_dates_2[key]
        temp_dict = {
            "date":key,
            "data_count":value,
            "morning_rush":extra_val[0],
            "free":extra_val[1],
            "evening_rush":extra_val[2]
        } 
        writer.writerow(temp_dict)
    dpd_writer.close()
            
    # for key,value in total_buses.iter():
    #     print(f"Bus {key} has {value} numbers of log in the dataset \n")

In [ ]:
# Extract categories and values
df = pd.read_csv(rcp)
categories = df['device_id']
group_a_values = df['morning_peak(7-10)']
group_b_values = df['free_hour']
group_c_values = df['evening_peak(4-7)']

# Set bar width and positions
bar_width = 0.2
index = np.arange(len(categories)) # Numerical positions for categories

# Create the figure and axes
fig, ax = plt.subplots(figsize=(20, 6))

# Plot each group of bars
bar1 = ax.bar(index - bar_width, group_a_values, bar_width, label='morning_peak(7-10)')
bar2 = ax.bar(index, group_b_values, bar_width, label='free_hour')
bar3 = ax.bar(index + bar_width, group_c_values, bar_width, label='evening_peak(4-7)')


ax.set_xlabel('Device_id')
ax.set_ylabel('No of datas')
ax.set_title('Mahanagar dataset 1.0')
ax.set_xticks(index)
ax.set_xticklabels(categories)
ax.legend()
plt.tight_layout() 
plt.show()

In [ ]:
# Extract categories and values
df = pd.read_csv(dpd)
categories = df['date']

group_a_values = df['morning_rush']
group_b_values = df['free']
group_c_values = df['evening_rush']
group_d_values = df["data_count"]
# Set bar width and positions
bar_width = 0.2
index = np.arange(len(categories)) # Numerical positions for categories

# Create the figure and axes
fig, ax = plt.subplots(figsize=(26, 6))

# Plot each group of bars
bar1 = ax.bar(index - bar_width, group_a_values, bar_width, label='morning_peak')
bar2 = ax.bar(index, group_b_values, bar_width, label='free_hour')
bar3 = ax.bar(index + bar_width, group_c_values, bar_width, label='evening_peak')
bar4 = ax.bar(index + bar_width*1.5, group_d_values, bar_width, label="total count") 

ax.set_xlabel('dates')
ax.set_ylabel("data_counts")
ax.set_title('Mahanagar dataset 1.1')
ax.set_xticks(index)
ax.set_xticklabels(categories)
ax.legend()
plt.tight_layout() 
plt.show()

In [ ]:

dict_info = {
    key:[] for key in buses
}
import math
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Returns the Haversine distance between two lat/lon points in meters.
    """
    R = 6371000  # Earth radius in meters

    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c
    
with open(save_path , "r") as file:
    
    take_date = dates_given[4]
    exp_yr , exp_mon,exp_day = make_int(take_date)
    print(exp_yr)
    reader = csv.DictReader(file)
    for line in reader:
        
        date_choose = line["fixTime"]
        date_time = date_choose.split()
        
        yr , mon,day = make_int(date_time[0])
        curr_hr ,curr_min , curr_sec = 0,0,0
        if yr == exp_yr and exp_mon == mon and exp_day == day:
            hr,min,sec = make_int(date_time[1])
            if hr==curr_hr and min==curr_min and curr_sec == sec:
                v = classify_hr(hr)
                discarded_entries[v]+=1
                continue
                
            #[(hr,min,sec),lat,lon]
            try:
                x = dict_info[int(line["deviceId"])][-1] 
                lat , lon = float(line["latitude"]) , float(line["longitude"])
            except:
                lat , lon = float(line["latitude"]) , float(line["longitude"])
            
                dict_info[int(line["deviceId"])].append([(hr,min,sec) , lat,lon])
                
                continue
            
            
            #calc. diffn in time 
            # time_taken = (abs(hr-prev_hr))*3600 + (abs(prev_min-min))*60 + (abs(sec-prev_sec))
            # dist = haversine_distance(lat,lon , prev_lat,prev_lon)
            dict_info[int(line["deviceId"])].append([(hr,min,sec) , lat,lon])
            curr_hr , curr_min,curr_sec = hr,min,sec
    file.close()     
    

In [ ]:
sorted_dict = {
    key:[[]] for key in buses
}
sorting_list = []
fixed_point = [27.678786911652914, 85.3494674406196] #koteshwor
def time_to_seconds(h, m, s):
    return h*3600 + m*60 + s


for key,value in dict_info.items():
    print(key)
    if value ==[]:
        continue
    sorted_values = sorted(value , key=lambda x:x[0] , reverse = False)
    prev_hr,prev_min ,prev_sec = 0,0,0
    prev_lat , prev_lon = 0,0
    total_distance_covered=0
    temp_list = [] 
    dist_counter = 0
    for x in sorted_values:
        print(x[0])
        hr ,min,sec=x[0]
        #print(prev_hr,prev_min , prev_sec , prev_lat,prev_lon)
        lat,lon = x[1],x[2]
        #print(key , hr,min,sec,lat,lon)
        
        if ((prev_hr ==0) and (prev_min==0) and (prev_sec==0)):
            prev_hr , prev_min,prev_sec = x[0]
            prev_lat , prev_lon = x[1],x[2]
            continue 
            #calc. diffn in time
        
        t1 = time_to_seconds(prev_hr, prev_min, prev_sec)
        t2 = time_to_seconds(hr, min, sec)

        time_taken = t2 - t1
        if time_taken < 10:
            v = classify_hr(hr)
            discarded_entries[v]+=1
            continue
        dist = haversine_distance(lat,lon , prev_lat,prev_lon)
        if dist < 100:# checks if the distance covered in 60 sec is less than 10 m and
            dist_counter+=1
            if dist_counter > 20:# removes last 20 entries ie data of 120 min if the bus is stationary
                v = classify_hr(hr)
                discarded_entries[v]+=20
                temp_list = temp_list[:-20]
                sorting_list = sorting_list[:-20]
        else :
            dist_counter = 0
        total_distance_covered +=dist

        fixed_dist = haversine_distance(lat,lon , fixed_point[0],fixed_point[1])
        sorting_list.append([(hr,min,sec) , lat,lon,time_taken , dist,total_distance_covered,fixed_dist])
        prev_hr , prev_min,prev_sec = x[0]
        prev_lat , prev_lon = x[1],x[2]
        if time_taken > 600:
            import random
            random_num = random.randint(000 , 1000000)
            with open(f"round_{key}_{random_num}.csv","w") as log_file:
                writer = csv.DictWriter(log_file , fieldnames=["time","lat","lon","time_taken","distance","total_distance","from_koteshwor"])
                writer.writeheader()
                for val in temp_list:
                    if val ==[]:
                        continue
                    dict_to_write = {
                        "time":val[0],
                        "lat":val[1],
                        "lon":val[2],
                        "time_taken":val[3],
                        "distance":val[4],
                        "total_distance":val[5],
                        "from_koteshwor":val[6]
                    }
                    writer.writerow(dict_to_write)
            log_file.close()
            temp_list = []
        else:
            temp_list.append([(hr,min,sec) , lat,lon,time_taken , dist,total_distance_covered,fixed_dist])
    #sorting_list = sorted(sorting_list,key = lambda x:x[0] , reverse = False)
    sorted_dict[key] = sorting_list 
    print(sorted_dict[key])
    sorting_list = []
with open(save_path_2,"w") as file: 
    writer = csv.DictWriter(file , fieldnames=["deviceId","time","lat","lon","time_taken","distance","total_distance","from_koteshwor"])
    writer.writeheader()
    for key,value in sorted_dict.items():
        for val in value:
            if val ==[] or val[3]==0:
                continue
            print("val is :" , val[0])
            dict_to_write = {
            "deviceId":key,
            "time":val[0],
            "lat":val[1],
            "lon":val[2],
            "time_taken":val[3],
            "distance":val[4],
            "total_distance":val[5],
            "from_koteshwor":val[6]
            }
            writer.writerow(dict_to_write)
    file.close()


In [ ]:
import matplotlib.pyplot as plt
import csv
import ast   # <-- To safely convert "(15, 18, 29)" into a real tuple

dict_ploting = {key: [[], []] for key in buses}

with open(save_path_2, "r") as file:
    reader = csv.DictReader(file)
    for line in reader:
        dev_id = int(line["deviceId"])

        # Convert string "(15, 18, 29)" into real tuple
        print(line["time"])
        t = ast.literal_eval(line["time"])
        print(t)
        dict_ploting[dev_id][0].append(t)

        # Total distance
        dist = float(line["total_distance"])
        dict_ploting[dev_id][1].append(dist)

# ---- PLOT ----
for bus_id, (time_list, distance_list) in dict_ploting.items():

    x = list(range(len(time_list)))   # equal interval x-axis

    plt.figure(figsize=(20, 8))
    plt.plot(x, distance_list, marker='o')

    plt.title(f"Distance vs Time for Bus {bus_id}")
    plt.xlabel("Equal Interval Points")
    plt.ylabel("Distance Covered")
    plt.grid(True)

    # Convert tuple to a readable label
    
    time_labels = [f"{h:02d}:{m:02d}:{s:02d}" for h, m, s in time_list]

    # Show max 20 x-tick labels to avoid clutter
    step = max(1, len(time_labels) // 20)
    plt.xticks(x[::step], time_labels[::step], rotation=45)

    plt.tight_layout()
    plt.show()
    plt.savefig(folder_img, dpi=300)


In [ ]:
import glob
import pandas as pd
import os
import shutil

# Reference longitude (you can adjust slightly — this is around Baneshwor / Old Baneshwor area)
fixed_lon = 85.32297540237306

# Find all round_*.csv files
round_files = glob.glob("round_*.csv")

print(f"Found {len(round_files)} round files. Processing...\n")

for file in round_files:
    try:
        df = pd.read_csv(file)

        # We only need at least 60 rows now
        if len(df) < 60:
            print(f"Skipping {file}: fewer than 60 rows (has {len(df)})")
            
            continue

        # Use 50th row (index 49) and 60th row (index 59)
        row50 = df.iloc[49]
        row60 = df.iloc[59]

        lat50, lon50 = row50['lat'], row50['lon']
        lat60, lon60 = row60['lat'], row60['lon']

        # Is the bus going north (increasing latitude)?
        going_north = lat60 > lat50

        # Which side of the reference longitude?
        side50 = "left"  if lon50 < fixed_lon else "right"
        side60 = "left"  if lon60 < fixed_lon else "right"

        # If it crossed the reference line between 50th and 60th → too ambiguous this early
        if side50 != side60:
            print(f"{file}: Crossed reference longitude between 50th and 60th row → skipping (ambiguous early segment)")
            continue

        # Core direction logic (Kathmandu Ring Road - view from above)
        if side50 == "left":   # Western half
            direction = "clockwise" if going_north else "counter_clockwise"
        else:                  # Eastern half
            direction = "counter_clockwise" if going_north else "clockwise"

        # Choose prefix
        prefix = "counter_clockwise_" if direction == "counter_clockwise" else "clockwise_"

        # Build new filename
        new_filename = prefix + os.path.basename(file)
        new_filepath = os.path.join(os.path.dirname(file) or '.', new_filename)

        # Avoid name collision
        if os.path.exists(new_filepath):
            base, ext = os.path.splitext(new_filename)
            counter = 1
            while os.path.exists(new_filepath):
                new_filepath = os.path.join(os.path.dirname(file) or '.', f"{base}_{counter}{ext}")
                counter += 1

        # Rename (move) the file
        shutil.move(file, new_filepath)

        print(f"Success: {os.path.basename(file)}")
        print(f"     → {os.path.basename(new_filepath)}")
        print(f"     Side: {side50} | 60th is {'NORTH' if going_north else 'SOUTH'} of 50th → {direction.upper()}\n")

    except Exception as e:
        print(f"Error processing {file}: {e}\n")

In [ ]:
BASE_STOPS_ORDER = [
    "koteshwor","airport", "gausala" , "chabhil" , "dhumbarahi" , "maharajgunj", "gangabu",
    "samakhushi" , "balaju" ,"banasthali", "swoyambhu", "sitapaila" ,"balkhu", "ekantakuna","satdobato", "gwarko","balkumari"
]

# Bus stations bounding boxes
bus_stations = {
    "anti-clockwise": {
        "koteshwor":[27.679261426232177, 85.34936846935956 , 27.680728503597233, 85.3495489426487],
        "airport":[27.70078540318246, 85.3533769530794,27.701573335528412, 85.35303133213452],#
        "gausala":[27.706379161514995, 85.34493560000823,27.70725888643062, 85.34418441639279],
        "chabhil":[27.717031239951016, 85.3463460556796,27.718204535772227, 85.34685091098906],
        #"dhumbarahi":[27.730953680990783, 85.34441299558648,27.732020831539714, 85.34419598186516],#
        #"maharajgunj":[27.73926959110684, 85.33815604032617,27.740494296415864, 85.33614465640026],#
        "gangabu":[27.73756021680712, 85.32456480569198,27.738041319788685, 85.32514445895883],
        "samakhushi": [27.734510272551045, 85.31315704526047,27.734988742644767, 85.31546713620789],
        "balaju": [27.726016065272873, 85.3035017969009,27.728129139328153, 85.30523648797792],
        "banasthali":[27.71883550527814, 85.2858452410448 , 27.719844381320847, 85.28726472663818],
        "swoyambhu":[27.71540016240659, 85.28353069074375,27.716986341444347, 85.28386474803716],
        "sitapaila": [27.707019479706727, 85.28256593334609,27.708527283182143, 85.28280436687375],
        #"balkhu":[27.684911593611844, 85.29708170941224,27.684903832051784, 85.29964109103348],
        #"ekantakuna":[27.668433982039407, 85.30668749053063,27.669623493257284, 85.30597299152524],#
        "satdobato":[27.658031146589305, 85.32347613583907,27.659581479957513, 85.32579715989439],
        "gwarko":[27.666433507619914, 85.33196568965492,27.667210717280035, 85.33248571687696],
        "balkumari":[27.671076985194357, 85.33969391131329,27.672139477723608, 85.34068226337071]
    },
    "clockwise": {
        "koteshwor":[27.679360243561582, 85.34941241032318,27.680401777864788, 85.34956815422122],
        "airport":[27.700269739797584, 85.35404763465152,27.700749790561282, 85.35380200362606],
        "gausala": [27.705844395215866, 85.34632228676749,27.70598189385746, 85.34821661918478],
        "chabhil":[27.71673824066052, 85.34652355945491,27.717243395322154, 85.34709193532692],
        #"dhumbarahi": [27.731781521213016, 85.34436394126764,27.733327061580734, 85.3435673251924],
        #"maharajgunj":[27.73937459407797, 85.33839402737382,27.740490229186403, 85.33639005980756],
        "gangabu":[27.73777682463683, 85.32466529203856,27.73859873484745, 85.32587780558195],
        "samakhushi":[27.734851265125926, 85.31257710156471,27.735064612431227, 85.31490388005503],
        "balaju": [27.72649827147075, 85.30373087544649,27.72855581557925, 85.30516384756554],
        "banasthali":[27.719298335255022, 85.28578533205231,27.720177454311088, 85.2874093632639],
        "swoyambhu":[27.716068321879543, 85.28346435660502,27.717078086320157, 85.28369225565842],
        "sitapaila": [27.70686115913291, 85.28219976757146,27.70857251696444, 85.28269366559307],
        #"balkhu":[27.68425789571755, 85.30065540395253,27.684345959864267, 85.30181369197074],
        #"ekantakuna":[27.667266839181096, 85.30718816078583,27.667906124565505, 85.30703180967741],
        "satdobato":[27.657961506465867, 85.3239280623327,27.659111317210236, 85.32546889945682],
        "gwarko":[27.665919477021834, 85.33192390175223,27.666869403657117, 85.33249964617663],
        "balkumari":[27.670955815839218, 85.33997817565354,27.672188974432935, 85.34113261637573]
    }
}

In [ ]:
import glob
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ast
from mpl_toolkits.axes_grid1 import make_axes_locatable
import traceback
# === Configuration and Setup ===

# Folders
FOLDER_HEATMAP_WAIT_TRAVEL = "heatmaps_wait_travel"
FOLDER_HEATMAP_CUMULATIVE = "heatmaps_cumulative"
FOLDER_PEAK_HEATMAPS = "peak_heatmaps"
os.makedirs(FOLDER_HEATMAP_WAIT_TRAVEL, exist_ok=True)
os.makedirs(FOLDER_HEATMAP_CUMULATIVE, exist_ok=True)
os.makedirs(FOLDER_PEAK_HEATMAPS, exist_ok=True)

# Peak hours (Ensure these variables are defined globally or passed in if running outside a script)


PEAK_MORNING = peak_morning
PEAK_EVENING = peak_evening

# Base stop order (used for indexing heatmaps)
BASE_STOPS_ORDER = [
    "koteshwor","airport", "gausala" , "chabhil" , "gangabu",
    "samakhushi" , "balaju" ,"banasthali", "swoyambhu", "sitapaila" ,"satdobato", "gwarko","balkumari"
]


DIRECTION_CONFIG = {
    "clockwise": {
        "pattern": "clockwise_*.csv",
        "dir_key": "clockwise",
        "stops_order": BASE_STOPS_ORDER
    },
    "counter_clockwise": {
        "pattern": "counter_clockwise_*.csv",
        "dir_key": "anti-clockwise",
        "stops_order": list(reversed(BASE_STOPS_ORDER))
    }
}

# === Helper Functions (Unchanged) ===
def time_to_seconds(t):
    """Converts a time tuple (h, m, s) to total seconds."""
    if isinstance(t, (tuple, list)) and len(t) == 3:
        return t[0]*3600 + t[1]*60 + t[2] 
    return 0
def time_to_hr(t):
    if isinstance(t, (tuple, list)) and len(t) == 3:
        return t[0]

    print("outttside if statement")
    return 0

def sec_to_time(sec):
    """Converts total seconds back to HH:MM:SS format."""
    sec = int(sec)
    h = sec // 3600
    m = (sec % 3600) // 60
    s = sec % 60
    return f"{h:02d}:{m:02d}:{s:02d}"


import math

def get_distance(lat1, lon1, lat2, lon2):
    # Simple distance approximation for performance
    return math.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

def get_stop(lat, lon, stations_dict, passed_stop, direction, threshold=0.003):
    """
    threshold: approx 500 meters in decimal degrees
    """
    # 1. First, check for an exact match (inside a box)
    for name, (s, w, n, e) in stations_dict.items():
        if __builtins__.min(s, n) <= lat <= max(s, n) and __builtins__.min(w, e) <= lon <= max(w, e):
            return name

    # 2. If no exact match, find the nearest stop from the expected sequence
    if passed_stop in BASE_STOPS_ORDER:
        curr_idx = BASE_STOPS_ORDER.index(passed_stop)
        
        # Check the next 2 stops in the sequence to see if we skipped one
        for i in range(1, 3):
            shift = i if direction == "anti-clockwise" else -i
            target_idx = (curr_idx + shift) % len(BASE_STOPS_ORDER)
            target_name = BASE_STOPS_ORDER[target_idx]
            
            # Get coordinates for this target stop
            s, w, n, e = stations_dict[target_name]
            center_lat, center_lon = (s + n) / 2, (w + e) / 2
            
            # Check if we are close enough to this stop
            if get_distance(lat, lon, center_lat, center_lon) < threshold:
                return target_name

    return f"In Transit (Passed {passed_stop})"
def classify_period(hr):
    if peak_morning[0]<=hr<=peak_morning[1]:
        return "morning"
    elif peak_evening[0]<=hr<=peak_evening[1]:
        return "evening"
    return "free"


# === MODIFIED HELPER FUNCTION: NEGATIVE VALUE CORRECTION ===

def correct_matrix_negatives(matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Applies the rule: if M[i, j] is negative, replace it with the absolute positive 
    value from M[j, i]. If M[j, i] is also not positive, set M[i, j] to 0.0.
    """
    matrix_corrected = matrix.copy()
    stops = matrix.index
    
    for i in range(len(stops)):
        for j in range(len(stops)):
            row_stop = stops[i]
            col_stop = stops[j]
            
            value = matrix_corrected.loc[row_stop, col_stop]
            
            if value < 0 and i != j: # Only check off-diagonal negative values
                # Get the reverse journey time (col_stop to row_stop)
                matrix_corrected.loc[col_stop, row_stop]  = abs(value)
                matrix_corrected.loc[row_stop , col_stop] = 0.0
                

    return matrix_corrected


In [ ]:


# === 1. Waiting & Travel heatmaps (per direction) - UNCHANGED ===
# (This section generates long, non-square heatmaps which are not subject to the i/j swap logic)
print("Generating Waiting and Travel Time Heatmaps (Dynamic Spectrum)...")
for dir_str, config in DIRECTION_CONFIG.items():
    files = glob.glob(config["pattern"])

    direction = "clockwise" if dir_str =="clockwise" else "anti-clockwise"
    if not files:
        print(f" No files found for {dir_str}")
        continue

    stops_order = config["stops_order"]
    stations = bus_stations[config["dir_key"]]
    
    travel_pairs = [f"{stops_order[i]}*to*{stops_order[(i+1) % len(stops_order)]}" for i in range(len(stops_order))]
    
    waiting_data = []
    travel_data = []

    for file in files:
        try:
            df = pd.read_csv(file)
            if isinstance(df['time'].iloc[0], str) and df['time'].iloc[0].startswith('('):
                df['time'] = df['time'].apply(ast.literal_eval)
            
            df['timestamp_sec'] = df['time'].apply(time_to_seconds)
            df["hr"]=df["time"].apply(time_to_hr)
            df['cum_time'] = df['time_taken'].cumsum().shift(fill_value=0)
            stops = []
            last_stop = None
            for _, row in df.iterrows():
                # def get_stop(lat, lon, stations_dict, passed_stop, direction, threshold=0.001):

                current_stop = get_stop(row['lat'], row['lon'], bus_stations['anti-clockwise'], last_stop , direction = direction)
                stops.append(current_stop)
                last_stop = current_stop
            
            df['stop'] = stops
            
            df['group_id'] = (df['stop'] != df['stop'].shift()).cumsum()
            groups = df.groupby('group_id')

            waiting_times = {stop: 0 for stop in stops_order}
            stop_arrivals = {}
            stop_departures = {}

            for group_id, group in groups:
                stop = group['stop'].iloc[0]
                if stop and pd.notna(stop) and len(group) > 1:
                    waiting = group['cum_time'].iloc[-1] - group['cum_time'].iloc[0]
                    waiting_times[stop] += waiting
                    
                    stop_arrivals[stop] = group['cum_time'].iloc[0]
                    stop_departures[stop] = group['cum_time'].iloc[-1]
            
            waiting_data.append({'file': os.path.basename(file), **waiting_times})
            
            stop_sequence = list(dict.fromkeys([g['stop'].iloc[0] for group_id, g in groups if g['stop'].iloc[0] and pd.notna(g['stop'].iloc[0])]))
            
            travel_times = {pair: 0 for pair in travel_pairs}
            for i in range(1, len(stop_sequence)):
                from_stop = stop_sequence[i-1]
                to_stop = stop_sequence[i]
                key = f"{from_stop}*to*{to_stop}"
                
                if key in travel_times and from_stop in stop_departures and to_stop in stop_arrivals:
                    travel = stop_arrivals[to_stop] - stop_departures[from_stop]
                    if travel > 0:
                        travel_times[key] = travel
            
            travel_data.append({'file': os.path.basename(file), **travel_times})
            
        except Exception as e:
            print(f" Error processing {file}: {e}")

    # Plotting Waiting Times (Now dynamic)
    if waiting_data:
        df_wait = pd.DataFrame(waiting_data).set_index('file').reindex(columns=stops_order).fillna(0)
        
        non_zero_vals = df_wait[df_wait > 0].values
        if len(non_zero_vals) > 0:
            wait_min = non_zero_vals.min()
            wait_max = non_zero_vals.max()
            mask = df_wait == 0 # Mask zero values
        else:
            wait_min, wait_max = 0, 1 # Fallback
            mask = df_wait == 0

        plt.figure(figsize=(12, len(files)*0.5 + 2))
        sns.heatmap(df_wait, annot=True, fmt=".0f", cmap="YlGnBu", 
                    cbar_kws={'label': 'Seconds'}, vmin=wait_min, vmax=wait_max, mask=mask)
        plt.title(f"Waiting Times at Stops - {dir_str.replace('_', ' ').title()} (Dynamic Spectrum)")
        plt.ylabel("Trip File")
        plt.xlabel("Stops")
        plt.tight_layout()
        plt.savefig(os.path.join(FOLDER_HEATMAP_WAIT_TRAVEL, f"waiting_{dir_str}.png"))
        plt.close()

    # Plotting Travel Times (Now dynamic)
    if travel_data:
        df_travel = pd.DataFrame(travel_data).set_index('file').reindex(columns=travel_pairs).fillna(0)
        
        non_zero_vals = df_travel[df_travel > 0].values
        if len(non_zero_vals) > 0:
            travel_min = non_zero_vals.min()
            travel_max = non_zero_vals.max()
            mask = df_travel == 0 # Mask zero values
        else:
            travel_min, travel_max = 0, 1 # Fallback
            mask = df_travel == 0

        plt.figure(figsize=(len(travel_pairs)*0.8, len(files)*0.5 + 2))
        sns.heatmap(df_travel, annot=True, fmt=".0f", cmap="OrRd", 
                    cbar_kws={'label': 'Seconds'}, vmin=travel_min, vmax=travel_max, mask=mask)
        plt.title(f"Travel Times Between Stops - {dir_str.replace('_', ' ').title()} (Dynamic Spectrum)")
        plt.ylabel("Trip File")
        plt.xlabel("Route Segment")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(FOLDER_HEATMAP_WAIT_TRAVEL, f"travel_{dir_str}.png"))
        plt.close()


# === 2. Cumulative journey heatmaps (per trip) - MODIFIED: FULL MATRIX + CORRECTION ===


In [ ]:

all_files = glob.glob("clockwise_*.csv") + glob.glob("counter_clockwise_*.csv")
print(f"\nGenerating cumulative journey heatmaps for {len(all_files)} trips (Full Matrix + Corrected)...")

for file_path in all_files:
    filename = os.path.basename(file_path)
    direction_name = "Counter-Clockwise" if "counter_clockwise" in filename else "Clockwise"
    dir_key = "anti-clockwise" if "counter_clockwise" in filename else "clockwise"
    route_order = BASE_STOPS_ORDER if direction_name == "Clockwise" else list(reversed(BASE_STOPS_ORDER))
    stations = bus_stations[dir_key]
    
    try:
        df = pd.read_csv(file_path)
        
        if isinstance(df['time'].iloc[0], str) and df['time'].iloc[0].startswith('('):
            df['time'] = df['time'].apply(ast.literal_eval)
        
        df['timestamp_sec'] = df['time'].apply(time_to_seconds)
        df["hr"]=df["time"].apply(time_to_hr)
        df['cum_time'] = df['time_taken'].cumsum().fillna(0)
        stops = []
        last_stop = None
        for _, row in df.iterrows():
            current_stop = get_stop(row['lat'], row['lon'], bus_stations['anti-clockwise'], last_stop , direction = dir_str)
            stops.append(current_stop)
            last_stop = current_stop

        df['stop'] = stops
        
        df['change'] = (df['stop'] != df['stop'].shift(1))
        df['segment'] = df['change'].cumsum()
        
        arrival_times = {}
        departure_times = {}
        arrival_clocks = {}

        for _, group in df.groupby('segment'):
            stop = group['stop'].iloc[0]
            if stop and pd.notna(stop):
                arrival_times[stop] = group['cum_time'].iloc[0]
                departure_times[stop] = group['cum_time'].iloc[-1]
                arrival_clocks[stop] = group['timestamp_sec'].iloc[0]
        
        visited_stops = [s for s in route_order if s in arrival_times]

        if len(visited_stops) < 13:
            continue

        matrix = pd.DataFrame(0.0, index=BASE_STOPS_ORDER, columns=BASE_STOPS_ORDER)

        # 1. Fill Diagonal with Waiting Time (Time spent at stop)
        for stop in visited_stops:
            if stop in arrival_times and stop in departure_times:
                waiting_time = (departure_times[stop] - arrival_times[stop]) / 60.0
                matrix.loc[stop, stop] = round(waiting_time, 1)

        # 2. Fill the FULL Matrix (Upper and Lower) with Cumulative Journey Time
        # The matrix will be symmetric, showing the time from any stop 'i' to any subsequent stop 'j'.
        for i in range(len(visited_stops)):
            for j in range(len(visited_stops)):
                if i == j: continue # Skip diagonal (waiting time already calculated)
                
                from_stop = visited_stops[i]
                to_stop = visited_stops[j]
                
                # We only calculate cumulative journey time from an earlier visited stop to a later visited stop.
                # If i < j (earlier stop to later stop):
                
                mins = (arrival_times[to_stop] - arrival_times[from_stop]) / 60.0
                if mins <0.0 :
                    continue
                matrix.loc[from_stop, to_stop] = round(mins, 1)
        
        # --- APPLY NEGATIVE VALUE CORRECTION HERE ---
        # This function will find any negative M[i,j] and try to replace it with M[j,i] if M[j,i] is positive.
        matrix = correct_matrix_negatives(matrix)
        
        # --- DYNAMIC SPECTRUM AND MASKING LOGIC ---
        valid_vals = matrix.values[matrix.values != 0.0]
        if valid_vals.size > 0:
            # Min/Max should still be dynamic based on the positive values after correction
            min_val = valid_vals.min()
            max_val = valid_vals.max()
        else:
            min_val, max_val = 0, 1 # Fallback
            
        mask = matrix == 0.0 
        
        fig = plt.figure(figsize=(15, 11))
        ax_main = plt.gca()
        sns.heatmap(matrix, annot=True, fmt=".1f", cmap="RdYlGn_r", linewidths=0.6, linecolor='gray',
                    cbar_kws={'label': 'Journey Time (minutes)'}, 
                    vmin=min_val, vmax=max_val, ax=ax_main, mask=mask)
        
        plt.text(0.5, 1.03, "Diagonal = Waiting Time at Stop (min) | Full Matrix Corrected for Negatives",
                 transform=ax_main.transAxes, fontsize=10, color='blue', ha='center')
        
        plt.title(f"Cumulative Journey Time Heatmap (Full Matrix)\n{filename}\n{direction_name} Route (Corrected)", fontsize=15, pad=30)
        plt.xlabel("To Stop (includes Waiting Time at destination stop) →")
        plt.ylabel("From Stop (includes Waiting Time at start stop) →")
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)

        # Timeline subplot (unchanged)
        divider = make_axes_locatable(ax_main)
        ax_timeline = divider.append_axes("bottom", size="25%", pad=0.7)
        times_sec = [arrival_clocks[s] for s in visited_stops]
        time_labels = [sec_to_time(t) for t in times_sec]
        
        ax_timeline.plot(times_sec, [0]*len(times_sec), 'o', color='blue',
                         markerfacecolor='white', markeredgewidth=2)
        ax_timeline.set_ylim(-1, 1)
        ax_timeline.set_yticks([])
        for spine in ['top', 'left', 'right']:
            ax_timeline.spines[spine].set_visible(False)
        ax_timeline.set_xlabel("Absolute Arrival Time (Clock)", fontsize=12, labelpad=15)
        
        for t, name in zip(times_sec, visited_stops):
            ax_timeline.text(t, 0.35, name.capitalize().replace("*", " "), rotation=40, ha='right', va='bottom', fontsize=10)
        for t, label in zip(times_sec, time_labels):
            ax_timeline.text(t, -0.5, label, rotation=40, ha='right', va='top', fontsize=9, color='gray')
            
        plt.tight_layout(rect=[0, 0.08, 1, 0.95])
        safe_name = os.path.splitext(filename)[0]
        output_path = os.path.join(FOLDER_HEATMAP_CUMULATIVE, f"cumulative_{safe_name}_full_corrected.png")
        plt.savefig(output_path, dpi=200, bbox_inches='tight')
        plt.close()
        print(f" Saved: cumulative_{safe_name}_full_corrected.png (Full Matrix Corrected)")
        
    except Exception as e:
        traceback.print_exc()
        print(f" ERROR processing {filename}: {e}")


In [ ]:

# === 3. Peak hour heatmaps (Average Journey Time + Average Waiting Time on Diagonal) - MODIFIED: FULL MATRIX + CORRECTION ===
print(f"\nGenerating peak hour analysis (Average Journey & Waiting Time with full matrix correction)...")
peak_records = []

# --- Collect Waiting Time Data (from=to) ---
# (Unchanged)
for dir_str, config in DIRECTION_CONFIG.items():
    files = glob.glob(config["pattern"])
    direction = "clockwise" if dir_str =="clockwise" else "anti-clockwise"

    stations = bus_stations[config["dir_key"]]
    for file in files:
        try:
            df = pd.read_csv(file)
            if isinstance(df['time'].iloc[0], str) and df['time'].iloc[0].startswith('('):
                df['time'] = df['time'].apply(ast.literal_eval)
                print(df["time"])
            df['timestamp_sec'] = df['time'].apply(time_to_seconds)
            stops = []
            last_stop = None
            for _, row in df.iterrows():
                current_stop =get_stop(row['lat'], row['lon'], bus_stations['anti-clockwise'], last_stop , direction = direction)
                stops.append(current_stop)
                last_stop = current_stop

            df['stop'] = stops
            df["hr"]=df["time"].apply(time_to_hr)
            trip_start_sec = df['timestamp_sec'].iloc[0]
            

            df['group_id'] = (df['stop'] != df['stop'].shift()).cumsum()
            df['cum_time'] = df['time_taken'].cumsum().shift(fill_value=0)
            groups = df.groupby('group_id')
            
            for group_id, group in groups:
                stop = group['stop'].iloc[0]
                if stop and pd.notna(stop) and len(group) > 1:
                    waiting = group['cum_time'].iloc[-1] - group['cum_time'].iloc[0]
                    if waiting >= 0:
                        period = classify_period(group["hr"])
                        peak_records.append({
                            "from": stop,
                            "to": stop,
                            "minutes": waiting / 60.0,
                            "period": period
                        })
        except Exception as e:
            pass



In [ ]:
# ASSUMPTIONS:
# - BASE_STOPS_ORDER, bus_stations, time_to_seconds, time_to_hr, get_stop, 
#   classify_period, peak_records (list), and all_files (list) are defined elsewhere.

# --- Collect Travel Time Data (from!=to) - CORRECTED: Collect only forward travel (i -> j, where j > i)
for file_path in all_files:
    try:
        df = pd.read_csv(file_path)

        if isinstance(df['time'].iloc[0], str) and df['time'].iloc[0].startswith('('):
            df['time'] = df['time'].apply(ast.literal_eval)

        # Determine the direction from the filename and set variables accordingly
        if "clockwise" in file_path:
            trip_direction = "clockwise"
            stations_dict = bus_stations["clockwise"]
        else:
            trip_direction = "anit-clockwise"
            stations_dict = bus_stations["anti-clockwise"]
            
        df['time_sec'] = df['time'].apply(time_to_seconds)
        df["hr"]=df["time"].apply(time_to_hr)
        df['cum_time'] = df['time_taken'].cumsum().fillna(0)
        
        stops = []
        last_stop = None
        for _, row in df.iterrows():
            
            current_stop = get_stop(row['lat'], row['lon'], bus_stations['anti-clockwise'], last_stop , direction = trip_direction)
            stops.append(current_stop)
            last_stop = current_stop

        df['stop'] = stops

        trip_start_sec = df['time_sec'].iloc[0]


        df['segment'] = (df['stop'] != df['stop'].shift()).cumsum()
        arrivals = df.dropna(subset=["stop"]).groupby("segment").first()
        stop_to_cum = dict(zip(arrivals["stop"], zip(arrivals["cum_time"] , arrivals["hr"])))

        # ASSUMPTION: BASE_STOPS_ORDER is the Counter-Clockwise order.
        route_order = BASE_STOPS_ORDER if trip_direction == "clockwise" else list(reversed(BASE_STOPS_ORDER))
        visited = [s for s in route_order if s in stop_to_cum]

        # Iterate over all unique pairs of visited stops (i, j)
        for i in range(len(visited)):
            # CRITICAL FIX: Ensure j is always greater than i to record forward movement in time.
            for j in range(i + 1, len(visited)): 
                
                from_stop = visited[i]
                to_stop = visited[j]

                cum_from, hr_from = stop_to_cum[from_stop]
                cum_to,   hr_to   = stop_to_cum[to_stop]

                minutes = (cum_to - cum_from) / 60.0
                
                # Safety check: If for any reason minutes are non-positive, skip the record.
                if minutes <= 0.0: 
                    continue 

                segment_hour = hr_from
                period = classify_period(segment_hour)
                
                # Record only valid, forward-travel segments
                peak_records.append({
                    "from": from_stop, 
                    "to": to_stop, 
                    "minutes": minutes, 
                    "period": period,
                    "direction": trip_direction
                })

    except Exception as e:
        traceback.print_exc()
        print(f" Peak analysis error {file_path}: {e}")

In [ ]:
# === Generate Average Peak Heatmaps + CSV Matrices (Directional Version) ===
# ASSUMPTIONS:
# - FOLDER_PEAK_HEATMAPS, BASE_STOPS_ORDER, correct_matrix_negatives (function) are defined elsewhere.
# - Libraries: import pandas as pd, import numpy as np, import matplotlib.pyplot as plt, import seaborn as sns, import os

if peak_records:
    df_peak = pd.DataFrame(peak_records)
    print(df_peak)
    
    # BASE_STOPS_ORDER is assumed to be the Counter-Clockwise order.
    
    def make_directional_peak_heatmap_and_csv(period, direction, title, png_filename, csv_filename):
        
        # --- FILTERING STEP ---
        subset = df_peak[
            (df_peak["period"] == period) &
            (df_peak["direction"] == direction)
        ]

        if subset.empty:
            print(f"No data for {period} period in {direction} direction.")
            return

        print(f"\n--- Processing {period.capitalize()} - {direction.capitalize()} ---")
        
        # 1. Compute stats and cap outliers
        stats = subset.groupby(['from', 'to'])['minutes'].agg(['mean', 'std', 'count']).reset_index()
        stats['std'] = stats['std'].fillna(0)
        
        # ACCURACY FIX: Enforce a non-negative lower bound (max(0, mean - 3*std))
        stats['lower_bound'] = (stats['mean'] - 3 * stats['std']).clip(lower=0) 
        stats['upper_bound'] = stats['mean'] + 3 * stats['std']

        subset_with_bounds = subset.merge(
            stats[['from', 'to', 'lower_bound', 'upper_bound']],
            on=['from', 'to'],
            how='left'
        )

        subset_clean = subset_with_bounds.copy()
        subset_clean['minutes_capped'] = subset_clean['minutes'].clip(
            lower=subset_clean['lower_bound'],
            upper=subset_clean['upper_bound']
        )

        print(f"{period.capitalize()} period, {direction.capitalize()} - Outliers capped (minimum 0 min)")

        # --- Base stop order for the current direction ---
        # BASE_STOPS_ORDER is assumed to be the Counter-Clockwise order.
        stops_order_for_matrix = BASE_STOPS_ORDER
        if direction == "clockwise":
             # Reverse order for clockwise route to ensure forward travel is displayed logically (top-left to bottom-right)
             stops_order_for_matrix = list(reversed(BASE_STOPS_ORDER))
        
        # 2. Compute average using capped values and reindex
        mat = subset_clean.pivot_table(
            values='minutes_capped',
            index='from',
            columns='to',
            aggfunc='mean'
        )
        mat = mat.reindex(index=stops_order_for_matrix, columns=stops_order_for_matrix).fillna(0.0)

        # 3. Apply correction (e.g., setting matrix diagonals/unvisited segments to 0)
        mat_corrected = correct_matrix_negatives(mat)

        # === Save CSV ===
        csv_name = f"{period}_rush_{direction}_matrix_3sigma_capped.csv"
        csv_path = os.path.join(FOLDER_PEAK_HEATMAPS, csv_name)
        mat_corrected.to_csv(csv_path)
        print(f"Saved CSV matrix: {csv_path}")

        # === Plot Heatmap ===
        
        # 4. Create the Mask based on Direction
        
        # Start with the existing mask (masking where data is 0.0)
        mask_zeros = mat_corrected == 0.0
        
        # Create a new, stronger directional mask
        if direction == "clockwise":
            # Clockwise (A->B->C) shows forward segments in the Upper Triangle. Mask the STRICT Lower Triangle.
            mask_directional = np.tril(np.ones_like(mat_corrected, dtype=bool), k=-1)
            
        elif direction == "counter_clockwise":
            # Counter-Clockwise (C->B->A) shows forward segments in the Lower Triangle. Mask the STRICT Upper Triangle.
            mask_directional = np.triu(np.ones_like(mat_corrected, dtype=bool), k=1)
            
        else:
             mask_directional = np.zeros_like(mat_corrected, dtype=bool)

        # Combine the directional mask with the mask for 0 values
        # We only mask the strict upper/lower triangle (k=-1 or k=1), preserving the diagonal (k=0) for wait times.
        mask = mask_zeros | mask_directional

        # Set min/max values
        valid_vals = mat_corrected.values[mat_corrected.values != 0.0]
        if valid_vals.size > 0:
            min_val = 0.0 # Time must start at 0
            max_val = valid_vals.max()
        else:
            min_val, max_val = 0, 1
            
        plt.figure(figsize=(15, 12))
        sns.heatmap(mat_corrected, annot=True, fmt=".1f", cmap="RdYlGn_r", linewidths=0.5, 
                    mask=mask, # Apply the combined mask here
                    cbar_kws={"label": "Avg Time (min)"}, vmin=min_val, vmax=max_val)
        
        plt.text(0.5, -0.05,
                 f"Diagonal = Avg Waiting Time | Direction: {direction.capitalize()} | Masked area represents irrelevant reverse segments.",
                 transform=plt.gca().transAxes, fontsize=10, color='blue', ha='center')
        
        plt.title(f"{title} - {direction.capitalize()} Route\n(Relevant segments displayed only)", fontsize=16)
        plt.xlabel("To Stop →")
        plt.ylabel("From Stop →")
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()

        png_name = f"{period}_rush_{direction}_heatmap_3sigma_capped.png"
        png_path = os.path.join(FOLDER_PEAK_HEATMAPS, png_name)
        plt.savefig(png_path, dpi=200, bbox_inches='tight')
        plt.close()
        print(f"Saved heatmap: {os.path.basename(png_path)}")

    # === Function Calls ===
    # Morning Rush
    make_directional_peak_heatmap_and_csv(
        period="morning", direction="clockwise",
        title="Morning Rush (07:00–10:00) Average Journey & Waiting Time",
        png_filename="morning_rush_clockwise_heatmap.png",
        csv_filename="morning_rush_clockwise_matrix.csv"
    )
    make_directional_peak_heatmap_and_csv(
        period="morning", direction="counter_clockwise",
        title="Morning Rush (07:00–10:00) Average Journey & Waiting Time",
        png_filename="morning_rush_counter_clockwise_heatmap.png",
        csv_filename="morning_rush_counter_clockwise_matrix.csv"
    )

    # Evening Rush
    make_directional_peak_heatmap_and_csv(
        period="evening", direction="clockwise",
        title="Evening Rush (17:00–19:00) Average Journey & Waiting Time",
        png_filename="evening_rush_clockwise_heatmap.png",
        csv_filename="evening_rush_clockwise_matrix.csv"
    )
    make_directional_peak_heatmap_and_csv(
        period="evening", direction="counter_clockwise",
        title="Evening Rush (17:00–19:00) Average Journey & Waiting Time",
        png_filename="evening_rush_counter_clockwise_heatmap.png",
        csv_filename="evening_rush_counter_clockwise_matrix.csv"
    )

    # Off-Peak Hours
    make_directional_peak_heatmap_and_csv(
        period="free", direction="clockwise",
        title="Off-Peak Hours Average Journey & Waiting Time",
        png_filename="free_time_clockwise_heatmap.png",
        csv_filename="free_time_clockwise_matrix.csv"
    )
    make_directional_peak_heatmap_and_csv(
        period="free", direction="counter_clockwise",
        title="Off-Peak Hours Average Journey & Waiting Time",
        png_filename="free_time_counter_clockwise_heatmap.png",
        csv_filename="free_time_counter_clockwise_matrix.csv"
    )

    print("\nAll directional peak heatmaps and CSV matrices successfully generated and saved in 'peak_heatmaps/' folder!")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np



# Print the values first
periods = ['Morning Rush', 'Free Time', 'Evening Rush']


# === Create Grouped Bar Chart ===
x = np.arange(len(periods))  # positions: 0, 1, 2
width = 0.35  # width of each bar

fig, ax = plt.subplots(figsize=(10, 6))

# Bars for Total and Discarded
bars1 = ax.bar(x - width/2, total_entries, width, label='Total Entries', color='skyblue', edgecolor='black')
bars2 = ax.bar(x + width/2, discarded_entries, width, label='Discarded Entries', color='salmon', edgecolor='black')

# Add numbers on top of each bar
def add_value_labels(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{int(height)}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 5),  # 5 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=11, fontweight='bold')

add_value_labels(bars1)
add_value_labels(bars2)

# Customize the plot
ax.set_xlabel('Time Period', fontsize=12)
ax.set_ylabel('Number of Entries', fontsize=12)
ax.set_title('Total vs Discarded Entries by Rush Hour Period', fontsize=15, pad=20)
ax.set_xticks(x)
ax.set_xticklabels(periods, fontsize=11)
ax.legend(fontsize=11)

# Add grid for better readability
ax.grid(axis='y', linestyle='--', alpha=0.7)
ax.set_axisbelow(True)

# Tight layout to prevent clipping
plt.tight_layout()

# Show the plot
plt.show()
